### Imports and function definitions

In [27]:
#This directory contains all the data needed for the module. It should be in the same directory as the notebook
dataFolder = "data"


import sys, os, shutil, errno, string, urllib


sys.path.append(( os.path.abspath( os.path.join(os.path.realpath("__file__"),  os.pardir, os.pardir) )))
import uvmodlib.v1 as uvmod

### USES Conf Templates

In [12]:
EarthCO2Template = """mesh
{  
    data  sphere ./modules/$folderName/sphere.obj
    cullRadius $cr
    glslVersion 330
    
    propertyCollection 
    {        
        __objectName__
        {           
            vec1f simUseTime  4500
        }
    }

    pass
    {
        useDataObject sphere
        shader
        {
            type defaultMeshShader
            {
                vertexShader   ./modules/$folderName/earthCO2.vs
                fragmentShader ./modules/$folderName/earthCO2.fs
                
                stateManagerVar __objectName__.Alpha        uv_alpha
                StateManagerVar __objectName__.simUseTime simUseTime
                
                
                glState
                {
                    UV_CULL_FACE_ENABLE false
                    #UV_CULL_MODE GL_FRONT
                    UV_BLEND_ENABLE true
                    UV_BLEND_FUNC GL_SRC_ALPHA GL_ONE_MINUS_SRC_ALPHA
                }
            }

        }    
    }
    
}"""

In [13]:
starTemplate='''mesh
{
    data SunData    ./modules/$folderName/MESA_1M_pre_ms_to_wd.raw

    glslVersion 330

    enableTypeCheck true
    creaseAngle 180.0
    cullRadius $cr


    propertyCollection
    {
        __objectName__
        {
            vec1f simCoronaTimeFac 4. #factor to multiply with time to determine rate of change in corona
            vec1f simSurfaceTimeFac 0.004 #factor to multiply with time to determine rate of change in  convective cells
            vec1f simBindRealtime 0. #bind the time of this model to the actual time in Uniview (0 == no, 1 == yes)
            vec1f simUseTime 4500. #if simBindReadtime == 0, then this is the time to use (Myr)
            vec1f simRealtimestart 0. #Myr
            vec1f simRealtimeend 12000. #Myr
            vec1f simShowstart 0.0
            vec1f simDtmin 0.01

#for star
            # alpha multiplier
            vec1f alpha 1

            #central temperature for exagerating the colors
            vec1f sTeff 5780.

            #factor to exagerate color (set to 1 for no exageration)
            vec1f Teffac 15

            # 0 cuts at the core. > 1 is no cut at all
            vec1f cutPlane 1
            
            # how quickly the corona disappears when going further from the star
            vec1f coronaFalloff 6.

            # 0 for no limb darkening
            vec1f limbDarkening 0.7

            vec1f brightness 1.5

            # controls the size of the convection cells
            vec1f convectionFrequency 1.

            # 0 means no convection
            vec1f convectionAmount 6.

            # controls the layers of the star interior
            # radius goes from 0 to 1
            vec3f ColorLayerConv 1 1 0
            vec3f ColorLayerRad  0 0 1
            vec3f ColorLayerHe  1 0 0



        }

    }


#####################################################
##Sun

    pass #Corona
    {
        passEnable true

        useDataObject SunData

        shader
        {
            type defaultMeshShader
            {
                parameter1f drawSun 0
                parameter1f drawCorona 1


                stateManagerVar __objectName__.alpha alpha
                stateManagerVar __objectName__.cutPlane cutPlane
                stateManagerVar __objectName__.coronaFalloff coronaFalloff
                stateManagerVar __objectName__.limbDarkening limbDarkening
                stateManagerVar __objectName__.brightness brightness
                stateManagerVar __objectName__.convectionFrequency convectionFrequency
                stateManagerVar __objectName__.convectionAmount convectionAmount
                stateManagerVar __objectName__.sTeff sTeff
                stateManagerVar __objectName__.Teffac Teffac
              
                stateManagerVar __objectName__.ColorLayerConv ColorLayerConv
                stateManagerVar __objectName__.ColorLayerRad ColorLayerRad
                stateManagerVar __objectName__.ColorLayerHe ColorLayerHe



                StateManagerVar __objectName__.simCoronaTimeFac simCoronaTimeFac
                StateManagerVar __objectName__.simSurfaceTimeFac simSurfaceTimeFac
                StateManagerVar __objectName__.simBindRealtime simBindRealtime
                StateManagerVar __objectName__.simUseTime simUseTime
                StateManagerVar __objectName__.simRealtimestart simRealtimestart  
                StateManagerVar __objectName__.simRealtimeend simRealtimeend 
                StateManagerVar __objectName__.simDtmin simdtmin

                
                texture bb ./modules/$folderName/bb.png
                {
                }

                vertexShader ./modules/$folderName/star_shader.vs
                fragmentShader ./modules/$folderName/star_shader.fs                
                geometryShader ./modules/$folderName/star_shader.gs



                glState
                {
                    UV_CULL_FACE_ENABLE false
                    UV_DEPTH_ENABLE false
                    UV_BLEND_ENABLE true
                    UV_WRITE_MASK_DEPTH true
                    UV_BLEND_FUNC GL_ONE GL_ONE_MINUS_SRC_ALPHA
                }
             }
        }
    }
    pass #Star
    {
        passEnable true

        useDataObject SunData

        shader
        {
            type defaultMeshShader
            {

                parameter1f drawSun 1
                parameter1f drawCorona 0

                stateManagerVar __objectName__.alpha alpha
                stateManagerVar __objectName__.cutPlane cutPlane
                stateManagerVar __objectName__.coronaFalloff coronaFalloff
                stateManagerVar __objectName__.limbDarkening limbDarkening
                stateManagerVar __objectName__.brightness brightness
                stateManagerVar __objectName__.convectionFrequency convectionFrequency
                stateManagerVar __objectName__.convectionAmount convectionAmount
                stateManagerVar __objectName__.sTeff sTeff
                stateManagerVar __objectName__.Teffac Teffac
              
                stateManagerVar __objectName__.ColorLayerConv ColorLayerConv
                stateManagerVar __objectName__.ColorLayerRad ColorLayerRad
                stateManagerVar __objectName__.ColorLayerHe ColorLayerHe



                StateManagerVar __objectName__.simCoronaTimeFac simCoronaTimeFac
                StateManagerVar __objectName__.simSurfaceTimeFac simSurfaceTimeFac
                StateManagerVar __objectName__.simBindRealtime simBindRealtime
                StateManagerVar __objectName__.simUseTime simUseTime
                StateManagerVar __objectName__.simRealtimestart simRealtimestart  
                StateManagerVar __objectName__.simRealtimeend simRealtimeend 
                StateManagerVar __objectName__.simDtmin simdtmin

                
                texture bb ./modules/$folderName/bb.png
                {
                }

                vertexShader ./modules/$folderName/star_shader.vs
                fragmentShader ./modules/$folderName/star_shader.fs                
                geometryShader ./modules/$folderName/star_shader.gs



                glState
                {
                    UV_CULL_FACE_ENABLE false
                    UV_DEPTH_ENABLE true
                    UV_BLEND_ENABLE true
                    UV_WRITE_MASK_DEPTH true
                    UV_BLEND_FUNC GL_SRC_ALPHA GL_ONE_MINUS_SRC_ALPHA
                }
             }
        }
    }

}

'''

### EarthCO2 Class

In [14]:
class EarthCO2():
    def __init__(self, object):
        self.object = object
        uvmod.Utility.ensurerelativepathexsists("sphere.obj",dataFolder)
        uvmod.Utility.ensurerelativepathexsists("earthCO2.vs",dataFolder)
        uvmod.Utility.ensurerelativepathexsists("earthCO2.fs",dataFolder)
        self.cr = 1000
    def generatemod(self):
        self.object.setgeometry(self.object.name+"Mesh.usesconf")
        return self.object.generatemod()
    def generatefiles(self, absOutDir, relOutDir):
        fileName = self.object.name+"Mesh.usesconf"
        s = string.Template(EarthCO2Template)
        f = open(absOutDir+"\\"+fileName, 'w')
        if f:
            f.write(s.substitute(folderName = relOutDir,
                                 cr = self.cr))
            f.close()
        uvmod.Utility.copyfoldercontents(os.getcwd()+"\\"+dataFolder, absOutDir)

## Star Class

In [15]:
class Star():
    def __init__(self, object):
        self.object = object
        uvmod.Utility.ensurerelativepathexsists("star_shader.gs",dataFolder)
        uvmod.Utility.ensurerelativepathexsists("star_shader.vs",dataFolder)
        uvmod.Utility.ensurerelativepathexsists("star_shader.fs",dataFolder)
        self.cr = 1000
    def generatemod(self):
        self.object.setgeometry("Star.usesconf")
        return self.object.generatemod()
    def generatefiles(self, absOutDir, relOutDir):
        fileName = "Star.usesconf"
        s = string.Template(starTemplate)
        f = open(absOutDir+"\\"+fileName, 'w')
        if f:
            f.write(s.substitute(folderName = relOutDir,
                                 cr = self.cr))
            f.close()
        uvmod.Utility.copyfoldercontents(os.getcwd()+"\\"+dataFolder, absOutDir)

### Object Instantiation

In [16]:
earth = EarthCO2(uvmod.OrbitalObject())
star = Star(uvmod.OrbitalObject())
generator = uvmod.Generator()
scene = uvmod.Scene()
modinfo = uvmod.ModuleInformation()

### Specify Settings and generate the module

In [24]:
earth.object.setcameraradius(10)
earth.object.setcoord("Earth")
earth.object.setname("EarthCO2")
earth.object.setguiname("/KavliLecture/Schrag/Earth CO2 Exterior")
earth.object.settargetradius(2000)
earth.object.setscalefactor(645.0)
earth.object.showatstartup(False)
earth.object.custom = "bin 9999"
earth.object.custom += """
    rotation
    {
        spice
        {
            frame		IAU_EARTH 
            reference	ECLIPJ2000
        }
    }"""

earth.cr = 10000

star.object.setcameraradius(50)
star.object.setcoord("SolarSystem")
star.object.setname("FaintYoungSun")
star.object.setguiname("/KavliLecture/Schrag/Faint Young Sun")
#star.object.settargetradius(500)
#star.object.setscalefactor(10.0)
star.object.showatstartup(False)



modinfo.setname("Faint Young Sun")
modinfo.setauthor("Aaron Geller<sup>1</sup> and Daniel Schrag<sup>2</sup><br />(1)Adler Planetarium,<br />(2)Harvard University")
modinfo.cleardependencies()
modinfo.adddependency("Earth")
modinfo.setdesc("A module to illustrate the change in the Sun's color and structure<br /> \
                from pre-main sequence through the red giant phase. <br /> \
                The data for the Sun comes from a MESA model. <br /> \
                This also includes an Earth module where the atmosphere color becomes<br /> \
                redder with increasing CO2.")
modinfo.setthumbnail("data/thumb.png")
modinfo.setversion("1.0")

generator.generate("FaintYoungSun",[],[earth,star],modinfo)
uvmod.Utility.senduvcommand(earth.object.name+".reload;")
uvmod.Utility.senduvcommand(star.object.name+".reload;")

## To reload

In [25]:
from config import Settings

In [26]:
uvmod.Utility.copyfoldercontents(os.getcwd()+"\\"+dataFolder, Settings.uvcustommodulelocation+'\\'+star.object.name)
uvmod.Utility.senduvcommand(star.object.name+".reload;")
uvmod.Utility.senduvcommand(earth.object.name+".reload;")